In [1]:
import numpy as np
import pandas as pd
import heapq as hp
import multiprocessing as mp
from time import sleep
from binance.client import Client

In [34]:
class StockMarket:

    def initialize_client(self): # тут мы инициализируем наш клиент чтобы он умел посылать запросики
        api_key = 'E5Qcm1IZZHToGacR1W108lpmTTjp5UUiScMJEbN7sgo2f4FYj7QQQkoGvnCYdDaj'
        api_secret = 'unvyAZ4ary0vCfGRlYHlA0yDRd82N5gzfYKK8J8iqaHdL3aMnKoI9kUZXZrVEZVN'
        self.client = Client(api_key, api_secret)
    
    
    def renew(self, currency_pair): # обновляем наш ордербук - на деле это будет происходить за счет DataCather, сейчас просто тест
        self.orders = self.client.get_order_book(symbol=currency_pair.upper())
        length = len(self.orders['asks'])
        self.orders['asks'] = [[float(self.orders['asks'][i][0]),
                                float(self.orders['asks'][i][1])]for i in range(length)]
        self.orders['bids'] = [[float(self.orders['bids'][i][0]),
                                float(self.orders['bids'][i][1])]for i in range(length)]
    
    
    def make_buy_order(self, currency_pair, amount): # эта функция вызывается когда мы хотим купить amount монеток по текущей цене
        spent = 0
        self.renew(currency_pair)
        for idx in range(len(self.orders['asks'])):
            if self.orders['asks'][idx][1] > amount:
                spent += self.orders['asks'][idx][0] * amount
                amount = 0
                break
            else:
                amount -= self.orders['asks'][idx][1]
                spent += self.orders['asks'][idx][1] * self.orders['asks'][idx][0]
        return spent

    
    def make_sell_order(self, currency_pair, amount): # если хотим продать amount монеток по рыночной цене - нам сюда
        capital = 0
        self.renew(currency_pair)
        for idx in range(len(self.orders['bids'])):
            if self.orders['bids'][idx][1] > amount:
                capital += self.orders['bids'][idx][0] * amount
                amount = 0
                break
            else:
                amount -= self.orders['bids'][idx][1]
                capital += self.orders['bids'][idx][1] * self.orders['bids'][idx][0]
        return capital
               
        
    # на вход поступает словарик {валютная пара: сколько хотим купить(если число отрицательное - продать)}
    def query(self, query_dict): 
        result = dict()
        for pair in query_dict:
            if query_dict[pair] > 0:
                result[pair] = [-self.make_buy_order(pair, query_dict[pair]), query_dict[pair]]
            elif query_dict[pair] < 0:
                result[pair] = [self.make_sell_order(pair, -query_dict[pair]), query_dict[pair]]
        return result # возвращаем словарик с изменениями в количестве валюты А и В
    
        
    def predict_price_movement(self, goes_up):
        # прогнозирование движения цены на следующей итерации стакана
        return

    
    def __init__ (self, ):
        self.initialize_client()
        
# Потсупает словарик с ключом - валютной парой и значением - сколько купить. Возвращаем словарик пар - на сколько
# изменилось количество первой и второй валюты. Напишу завтра

In [35]:
market = StockMarket()

In [36]:
market.query({'btcusdt': 3, 'ltcbtc': -5})

{'btcusdt': [-28874.097798879997, 3], 'ltcbtc': [0.03856, -5]}